# Comparison between different comb (new version including adt correction)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir
from histlib.matchup import _data_var, _stress_var, _aviso_var

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=10, processes=10, walltime='04:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.66:49593/status,
Dashboard: http://10.148.1.66:49593/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.66:45037,Workers: 0
Dashboard: http://10.148.1.66:49593/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
DL =25*1e3 #meters
DT = 1800.0 #seconds

In [9]:
def global_ms_drifter_sat_year(dsms):
    ds = (((dsms*dsms.nb_coloc).sum('drifter_sat_year'))/(dsms.nb_coloc.sum('drifter_sat_year'))).drop('nb_coloc')
    for v in dsms.keys():
        if v != 'nb_coloc':
            ds[v].attrs=dsms[v].attrs
    return ds

In [11]:
dsd = xr.open_dataset(os.path.join(zarr_dir+'_ok',f'ms_{int(DL//1000)}_{DT}_drogued.zarr'))
gps = [l for l in dsd.drifter_sat_year.values if 'gps' in l]
ds = global_ms_drifter_sat_year(dsd.sel(drifter_sat_year=gps)).persist()

In [22]:
ds[[v for v in ds if 'sum' in v]]

<xarray.Dataset>
Dimensions:                                                                         ()
Data variables:
    sum_aviso__0__adt__es_cstrio_z15__alti_x                                        float64 ...
    sum_aviso__0__adt__es_cstrio_z15__alti_y                                        float64 ...
    sum_aviso__0__adt__es_cstrio_z15__drifter_x                                     float64 ...
    sum_aviso__0__adt__es_cstrio_z15__drifter_y                                     float64 ...
    sum_co__0__adt_filtered__es_cstrio_z15__alti_x                                  float64 ...
    sum_co__0__adt_filtered__es_cstrio_z15__drifter_x                               float64 ...
    sum_co__0__adt_filtered_ocean_tide__es_cstrio_z15__alti_x                       float64 ...
    sum_co__0__adt_filtered_ocean_tide__es_cstrio_z15__drifter_x                    float64 ...
    sum_co__0__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__alti_x         float64 ...
    sum_co__0__adt_filtered_ocean_tide_internal_tide__es_cstrio_z15__drifter_x      float64 ...
    sum_co__0__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__alti_x     float64 ...
    sum_co__0__adt_filtered_ocean_tide_internal_tide_dac__es_cstrio_z15__drifter_x  float64 ...

In [23]:
cluster.close()